In [2]:
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from keras.layers.normalization import BatchNormalization
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import gc
# fix random seed for reproducibility
np.random.seed(7)

In [3]:
dt_train = pd.read_csv("../../data/MelbDatathon2017/pythonData/trainData_ivan.csv")
dt_test = pd.read_csv("../../data/MelbDatathon2017/pythonData/testData_ivan.csv")

In [4]:
y = dt_train.pop("Target")
X = dt_train

In [5]:
y_test = dt_test.pop("Target")
X_test = dt_test
X_test = X_test.loc[:, ~X_test.columns.str.contains('^Unnamed')]

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.99, random_state = 42)

In [7]:
id_train = X_train.pop("Patient_ID")
id_valid = X_valid.pop("Patient_ID")
id_test = X_test.pop("Patient_ID")

In [8]:
del X, y, dt_train, dt_test; gc.collect()

76

In [9]:
X_train.shape

(2792, 867)

In [10]:
X_train[np.isnan(X_train)] = 0
X_valid[np.isnan(X_valid)] = 0
X_test[np.isnan(X_test)] = 0

In [11]:
X_train = X_train.as_matrix()
X_valid = X_valid.as_matrix()
X_test = X_test.as_matrix()

y_train = np.array(y_train)
y_valid = np.array(y_valid)
y_test = np.array(y_test)

In [12]:
# create model
model = Sequential()
model.add(Dense(32, input_dim=867, kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(32, kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [13]:
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
# Fit the model
model.fit(X_train, y_train, epochs=2500, batch_size=200, validation_data=(X_valid, y_valid), verbose = 0)

In [20]:
predictions = model.predict_proba(X_valid, verbose = 0)

In [21]:
min(predictions)

array([ 0.], dtype=float32)

In [31]:
from sklearn import metrics
metrics.roc_auc_score(y_valid, predictions[:,0])

0.95365447771762324

In [23]:
pred_test = model.predict_proba(X_test, verbose = 0)

In [30]:
pred_test[:,0]

array([  1.92553099e-36,   2.51354965e-22,   6.23120897e-08, ...,
         2.96855940e-09,   8.63396671e-16,   9.98023748e-01], dtype=float32)

In [32]:
dt_submit = pd.DataFrame({"Patient_ID": id_test, "Diabetes": pred_test[:,0]})

In [33]:
dt_submit[["Patient_ID", "Diabetes"]].to_csv("../../data/MelbDatathon2017/submission/NN_1.csv", index = False)